In [3]:
# So HF saves cache to RunPod's persistent volume
import os
os.environ["HF_HOME"] = "/workspace/cache/"
os.environ["TRANSFORMERS_CACHE"] = "/workspace/cache/"

In [4]:
import torch
import transformer_lens.loading_from_pretrained as loading
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer

torch.set_grad_enabled(False)

In [5]:
# Example of loading from hf then tlens
dtype = torch.float32
weights_source = "NousResearch/Llama-2-7b-hf"
tlens_arch = "Llama-2-7b-hf"
# weights_source = "NousResearch/Llama-2-7b-chat-hf"
# tlens_arch = "Llama-2-7b-chat-hf"

In [6]:
# Load from hf
tokenizer = AutoTokenizer.from_pretrained(weights_source)
hf_model = AutoModelForCausalLM.from_pretrained(weights_source, torch_dtype=dtype)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/workspace/mamba-envs/mats/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/workspace/mamba-envs/mats/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
# Load from hf into tlens
# cfg = loading.get_pretrained_model_config(tlens_arch, torch_type=dtype)
cfg = loading.get_pretrained_model_config(tlens_arch, dtype=torch.float16, device="cuda")
tl_model = HookedTransformer(cfg, tokenizer=tokenizer)
state_dict = loading.get_pretrained_state_dict(tlens_arch, cfg, hf_model=hf_model, dtype=torch.float16)
tl_model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['blocks.0.attn.mask', 'blocks.0.attn.IGNORE', 'blocks.0.attn.rotary_sin', 'blocks.0.attn.rotary_cos', 'blocks.1.attn.mask', 'blocks.1.attn.IGNORE', 'blocks.1.attn.rotary_sin', 'blocks.1.attn.rotary_cos', 'blocks.2.attn.mask', 'blocks.2.attn.IGNORE', 'blocks.2.attn.rotary_sin', 'blocks.2.attn.rotary_cos', 'blocks.3.attn.mask', 'blocks.3.attn.IGNORE', 'blocks.3.attn.rotary_sin', 'blocks.3.attn.rotary_cos', 'blocks.4.attn.mask', 'blocks.4.attn.IGNORE', 'blocks.4.attn.rotary_sin', 'blocks.4.attn.rotary_cos', 'blocks.5.attn.mask', 'blocks.5.attn.IGNORE', 'blocks.5.attn.rotary_sin', 'blocks.5.attn.rotary_cos', 'blocks.6.attn.mask', 'blocks.6.attn.IGNORE', 'blocks.6.attn.rotary_sin', 'blocks.6.attn.rotary_cos', 'blocks.7.attn.mask', 'blocks.7.attn.IGNORE', 'blocks.7.attn.rotary_sin', 'blocks.7.attn.rotary_cos', 'blocks.8.attn.mask', 'blocks.8.attn.IGNORE', 'blocks.8.attn.rotary_sin', 'blocks.8.attn.rotary_cos', 'blocks.9.attn.mask', 'blocks.9.attn.IGNORE', 'blo

In [54]:
tokens = tokenizer("Why did the chicken cross the road?", return_tensors="pt")["input_ids"]
hf_logits = hf_model(tokens).logits
tl_logits = tl_model(tokens)

torch.allclose(hf_logits.cpu(), tl_logits.cpu(), atol=1.2e-5)

True